In [122]:
import pandas as pd
import os
import numpy as np

msr_columns=['Timestamp','Hostname','DiskNumber','Type','Offset','Size','ResponseTime']
df = pd.read_csv('/home/yun/Trace/origin/MSR/hm_0.csv', names=msr_columns, header=None)
print(np.shape(df))
df.head()

(3993316, 7)


,Timestamp,Hostname,DiskNumber,Type,Offset,Size,ResponseTime
0,128166385295514663,hm,0,Write,9056014336,2048,2833
1,128166385295514878,hm,0,Write,11855351808,512,2617
2,128166385295515175,hm,0,Write,5548077056,4096,2320
3,128166385301451520,hm,0,Write,3163095040,40960,3438
4,128166385301453920,hm,0,Write,3154132992,4096,1037


In [123]:
df_write = df.loc[df['Type']=='Write']  # write만 분류
print(np.shape(df_write))
df_write.head()

(2575568, 7)


,Timestamp,Hostname,DiskNumber,Type,Offset,Size,ResponseTime
0,128166385295514663,hm,0,Write,9056014336,2048,2833
1,128166385295514878,hm,0,Write,11855351808,512,2617
2,128166385295515175,hm,0,Write,5548077056,4096,2320
3,128166385301451520,hm,0,Write,3163095040,40960,3438
4,128166385301453920,hm,0,Write,3154132992,4096,1037


In [124]:
df_use = df_write.loc[:,['Offset', 'Size']] # 필요한 열만 추출
df_use

,Offset,Size
0,9056014336,2048
1,11855351808,512
2,5548077056,4096
3,3163095040,40960
4,3154132992,4096
...,...,...
3993286,13575007232,512
3993288,13575007232,512
3993290,13575007232,512
3993291,13575007232,512


In [125]:
df_use.reset_index(drop=True, inplace=True)
df_use.reset_index(inplace=True)
df_use.rename(columns={'index':'PrevDT'}, inplace=True)

In [126]:
df_use

,PrevDT,Offset,Size
0,0,9056014336,2048
1,1,11855351808,512
2,2,5548077056,4096
3,3,3163095040,40960
4,4,3154132992,4096
...,...,...,...
2575563,2575563,13575007232,512
2575564,2575564,13575007232,512
2575565,2575565,13575007232,512
2575566,2575566,13575007232,512


In [127]:
groups = df_use.groupby('Offset')
df_use['first'] = groups.PrevDT.transform('first')
df_use['NextDT'] = groups['PrevDT'].shift(-1)


In [112]:
df_use.loc[df_use['Offset']==11855351808]

,PrevDT,Offset,Size,first,NextDT
1,1,11855351808,512,1,1735.0
1735,1735,11855351808,2048,1,1813.0
1813,1813,11855351808,2048,1,1818.0
1818,1818,11855351808,4096,1,999189.0
999189,999189,11855351808,4096,1,NaN


In [128]:
df_use['first'] = df_use['first'] + 2575568
df_use

,PrevDT,Offset,Size,first,NextDT
0,0,9056014336,2048,2575568,490.0
1,1,11855351808,512,2575569,1735.0
2,2,5548077056,4096,2575570,999849.0
3,3,3163095040,40960,2575571,681077.0
4,4,3154132992,4096,2575572,28.0
...,...,...,...,...,...
2575563,2575563,13575007232,512,2603411,2575564.0
2575564,2575564,13575007232,512,2603411,2575565.0
2575565,2575565,13575007232,512,2603411,2575566.0
2575566,2575566,13575007232,512,2603411,NaN


In [129]:
df_final = df_use.fillna(method='ffill', axis=1)

In [132]:
df_final.drop(['first'], axis=1, inplace=True)

In [133]:
df_final

,PrevDT,Offset,Size,NextDT
0,0.0,9.056014e+09,2048.0,490.0
1,1.0,1.185535e+10,512.0,1735.0
2,2.0,5.548077e+09,4096.0,999849.0
3,3.0,3.163095e+09,40960.0,681077.0
4,4.0,3.154133e+09,4096.0,28.0
...,...,...,...,...
2575563,2575563.0,1.357501e+10,512.0,2575564.0
2575564,2575564.0,1.357501e+10,512.0,2575565.0
2575565,2575565.0,1.357501e+10,512.0,2575566.0
2575566,2575566.0,1.357501e+10,512.0,2603411.0


In [135]:
df_final = df_final.astype(int)

In [136]:
df_final.dtypes

PrevDT    int64
Offset    int64
Size      int64
NextDT    int64
dtype: object

In [137]:
df_final

,PrevDT,Offset,Size,NextDT
0,0,9056014336,2048,490
1,1,11855351808,512,1735
2,2,5548077056,4096,999849
3,3,3163095040,40960,681077
4,4,3154132992,4096,28
...,...,...,...,...
2575563,2575563,13575007232,512,2575564
2575564,2575564,13575007232,512,2575565
2575565,2575565,13575007232,512,2575566
2575566,2575566,13575007232,512,2603411


In [139]:
df_final.to_csv("/home/yun/Trace/DT/hm_0.csv", index=False)